In [1]:
# nb_gen_features.ipynb - Generacion de features para modelos ML
import pandas as pd
import numpy as np
from pathlib import Path
from datetime import date
import logging

# Configurar rutas
BASE = Path("D:/trading")
HIST_PATH = BASE / "data" / "historic"
FEA_PATH = BASE / "data" / "features"
LOG_PATH = BASE / "logs" / f"gen_features_{date.today().isoformat()}.log"

# Crear carpeta de salida si no existe
FEA_PATH.mkdir(parents=True, exist_ok=True)

# Configurar log
logging.basicConfig(filename=LOG_PATH, level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")
print("Rutas configuradas correctamente.")


Rutas configuradas correctamente.


In [2]:
archivos = list(HIST_PATH.glob("*.parquet"))
print(f"Total de archivos a procesar: {len(archivos)}")
logging.info(f"Total de archivos a procesar: {len(archivos)}")


Total de archivos a procesar: 50


In [3]:
def calcular_features(df):
    df = df.copy()
    df["retorno"] = df["close"].pct_change()
    df["volatilidad"] = df["retorno"].rolling(window=10).std()
    df["sma_10"] = df["close"].rolling(window=10).mean()
    df["sma_20"] = df["close"].rolling(window=20).mean()
    df["ema_10"] = df["close"].ewm(span=10).mean()
    df["ema_20"] = df["close"].ewm(span=20).mean()
    df["rsi_14"] = calcular_rsi(df["close"], 14)
    df["macd"] = df["close"].ewm(span=12).mean() - df["close"].ewm(span=26).mean()
    df["macd_signal"] = df["macd"].ewm(span=9).mean()
    return df

def calcular_rsi(series, period):
    delta = series.diff()
    gain = np.where(delta > 0, delta, 0)
    loss = np.where(delta < 0, -delta, 0)
    avg_gain = pd.Series(gain).rolling(window=period).mean()
    avg_loss = pd.Series(loss).rolling(window=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi


In [4]:
for archivo in archivos:
    symbol = archivo.stem.upper()
    try:
        df = pd.read_parquet(archivo)
        df["fecha"] = pd.to_datetime(df["datetime"], errors="coerce")
        df["simbolo"] = symbol
        df_features = calcular_features(df)
        salida = FEA_PATH / f"{symbol}_features.parquet"
        df_features.to_parquet(salida, index=False)
        logging.info(f"OK - {symbol} - {len(df_features)} filas procesadas.")
        print(f"{symbol} listo.")
    except Exception as e:
        logging.error(f"ERROR - {symbol} - {e}")
        print(f"{symbol} fallo.")


AEP listo.
AGNCO listo.
AGNCP listo.
BAC listo.
BTSGU listo.
BX listo.
C listo.
CCI listo.
CFG listo.
COP listo.
CRBG listo.
CTVA listo.
DELL listo.
DGX listo.
DUK listo.
ECCF listo.
ED listo.
EPD listo.
EQR listo.
EW listo.
EXE listo.
FITBO listo.
FWONA listo.
FWONK listo.
HBANL listo.
HBANM listo.
HBANP listo.
HPE listo.
IRM listo.
KHC listo.
KKRS listo.
LPLA listo.
MCHPP listo.
MORN listo.
MRNA listo.
OWL listo.
PCTY listo.
PEN listo.
PRMB listo.
PSTG listo.
RKT listo.
RPRX listo.
SOJC listo.
SOLV listo.
SYM listo.
TBB listo.
TEM listo.
TPG listo.
VLYPN listo.
WPC listo.


In [9]:
# Limpieza de NaN y log detallado
import pandas as pd
from pathlib import Path
from datetime import datetime

# Ruta base
features_dir = Path("D:/trading/data/features/")
log_drop_path = Path("D:/trading/logs/log_drop_features.csv")

# Inicializar log
registros_log = []

for archivo in sorted(features_dir.glob("*.parquet")):
    simbolo = archivo.stem.replace("_features", "")
    df = pd.read_parquet(archivo)
    total = len(df)
    nulos = df.isna().sum().sum()

    if nulos > 0:
        columnas_nulas = df.columns[df.isna().any()].tolist()
        df_limpio = df.dropna()
        df_limpio.to_parquet(archivo, index=False)
        estado = "DROP"
        detalle = f"{len(df_limpio)} filas -> columnas nulas: {','.join(columnas_nulas)}"
    else:
        estado = "OK"
        detalle = f"{total} filas sin nulos"

    registros_log.append({
        "datetime": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
        "symbol": simbolo,
        "status": estado,
        "mensaje": detalle
    })

# Guardar log
pd.DataFrame(registros_log).to_csv(log_drop_path, index=False)
print(f"Proceso finalizado. Log: {log_drop_path.name}")


Proceso finalizado. Log: log_drop_features.csv


In [10]:
# ✅ Validar archivos de features para entrenamiento
import pandas as pd
from pathlib import Path

FEATURES_DIR = Path("D:/trading/data/features")
columnas_minimas = {"fecha", "simbolo", "close"}

archivos = list(FEATURES_DIR.glob("*.parquet"))
errores = []

print(f"Total de archivos encontrados: {len(archivos)}\n")

for file in archivos:
    try:
        df = pd.read_parquet(file)
        if not columnas_minimas.issubset(df.columns):
            errores.append(f"{file.name} columnas faltantes: {columnas_minimas - set(df.columns)}")
        elif df.isna().sum().sum() > 0:
            errores.append(f"{file.name} contiene valores nulos")
        elif df.empty:
            errores.append(f"{file.name} esta vacio")
    except Exception as e:
        errores.append(f"{file.name} error de lectura: {e}")

if errores:
    print("Archivos con errores encontrados:\n")
    for err in errores:
        print(" -", err)
else:
    print("✅ Todos los archivos son validos para entrenamiento.")


Total de archivos encontrados: 50

✅ Todos los archivos son validos para entrenamiento.


In [12]:
import pandas as pd
from pathlib import Path

# Ruta de archivos de features
FEATURES_DIR = Path("D:/trading/data/features/")
archivos = list(FEATURES_DIR.glob("*_features.parquet"))

# Mostrar cantidad de filas validas por simbolo
print("Resumen de filas por simbolo:\n")
resumen = []
for file in archivos:
    simbolo = file.stem.replace("_features", "")
    df = pd.read_parquet(file)
    resumen.append((simbolo, len(df)))

# Ordenar y mostrar
resumen_df = pd.DataFrame(resumen, columns=["simbolo", "filas"]).sort_values(by="filas", ascending=False)
display(resumen_df)

# Inspeccionar simbolo especifico: DELL
dell_file = FEATURES_DIR / "DELL_features.parquet"
df_dell = pd.read_parquet(dell_file)

print(f"\nDELL - tipos de datos:")
print(df_dell.dtypes)

print(f"\nDELL - primeras filas:")
display(df_dell.head())


Resumen de filas por simbolo:



,simbolo,filas
0,AEP,4981
11,C,4981
38,PRMB,4981
33,MORN,4981
28,IRM,4981
19,EW,4981
18,EQR,4981
17,EPD,4981
16,ED,4981
14,DUK,4981



DELL - tipos de datos:
datetime       datetime64[ns]
open                  float64
high                  float64
low                   float64
close                 float64
volume                  int64
fecha          datetime64[ns]
simbolo                object
retorno               float64
volatilidad           float64
sma_10                float64
sma_20                float64
ema_10                float64
ema_20                float64
rsi_14                float64
macd                  float64
macd_signal           float64
dtype: object

DELL - primeras filas:


,datetime,open,high,low,close,volume,fecha,simbolo,retorno,volatilidad,sma_10,sma_20,ema_10,ema_20,rsi_14,macd,macd_signal
0,2016-09-14,13.58035,13.64489,13.47089,13.56912,17208658,2016-09-14,DELL,-0.003092,0.033589,13.197266,12.858529,13.369298,13.145504,59.595253,0.222123,0.150991
1,2016-09-15,13.50457,13.75996,13.47651,13.72067,17208302,2016-09-15,DELL,0.011169,0.033008,13.315417,12.941179,13.434143,13.207911,61.852847,0.229118,0.166762
2,2016-09-16,13.64770,13.74873,13.50457,13.60280,17786615,2016-09-16,DELL,-0.008591,0.031289,13.454897,13.021444,13.465183,13.250196,59.214075,0.223897,0.178274
3,2016-09-19,13.53825,13.72067,13.51580,13.68418,16264400,2016-09-19,DELL,0.005983,0.030306,13.617109,13.095253,13.505399,13.296124,58.482715,0.222722,0.187216
4,2016-09-20,13.66454,13.73470,13.51860,13.67576,12000986,2016-09-20,DELL,-0.000615,0.026137,13.709160,13.166536,13.536627,13.335879,63.378113,0.218602,0.193523


In [6]:
# Celda para mostrar el log completo del proceso de generacion de features
from pathlib import Path
from datetime import date

# Ruta del log
LOG_PATH = Path("D:/trading/logs") / f"gen_features_{date.today().isoformat()}.log"

# Leer y mostrar contenido
if LOG_PATH.exists():
    with open(LOG_PATH, "r", encoding="utf-8") as f:
        contenido = f.read()
    print(f"Contenido del log generado: {LOG_PATH.name}\n")
    print(contenido)
else:
    print("Log no encontrado.")


Contenido del log generado: gen_features_2025-05-27.log

2025-05-27 22:22:22,960 - INFO - Total de archivos a procesar: 50
2025-05-27 22:22:23,214 - INFO - OK - AEP - 5000 filas procesadas.
2025-05-27 22:22:23,255 - INFO - OK - AGNCO - 1422 filas procesadas.
2025-05-27 22:22:23,294 - INFO - OK - AGNCP - 1334 filas procesadas.
2025-05-27 22:22:23,377 - INFO - OK - BAC - 5000 filas procesadas.
2025-05-27 22:22:23,419 - INFO - OK - BTSGU - 333 filas procesadas.
2025-05-27 22:22:23,479 - INFO - OK - BX - 4511 filas procesadas.
2025-05-27 22:22:23,530 - INFO - OK - C - 5000 filas procesadas.
2025-05-27 22:22:23,581 - INFO - OK - CCI - 5000 filas procesadas.
2025-05-27 22:22:23,616 - INFO - OK - CFG - 2684 filas procesadas.
2025-05-27 22:22:23,683 - INFO - OK - COP - 5000 filas procesadas.
2025-05-27 22:22:23,709 - INFO - OK - CRBG - 676 filas procesadas.
2025-05-27 22:22:23,738 - INFO - OK - CTVA - 1510 filas procesadas.
2025-05-27 22:22:23,790 - INFO - OK - DELL - 2206 filas procesadas.
20

In [7]:
from pathlib import Path

# Listar archivos en la carpeta de logs
log_dir = Path("D:/trading/logs")
log_files = sorted(log_dir.glob("*.log"))

print(f"Se encontraron {len(log_files)} archivos de log:\n")
for f in log_files:
    print("-", f.name)


Se encontraron 17 archivos de log:

- breakout_volumen.log
- cruce_medias.log
- ema_9_21_cruce.log
- ema_pullback.log
- gap_open_strategy.log
- gen_features_2025-05-27.log
- inside_bar_breakout.log
- ma_envelope_reversals.log
- macd_cruce.log
- macd_hist_reversal.log
- mov_avg.log
- rsi_divergencia.log
- rsi_reversion.log
- shp_2025-05-27.log
- sma_10_50_cruce.log
- soporte_resistencia.log
- volatilidad_contraria.log


In [11]:

import pandas as pd
from pathlib import Path

# Ruta del log generado
log_file = Path("D:/trading/logs/gen_features_2025-05-27.log")

# Leer y estructurar
with open(log_file, encoding="utf-8") as f:
    lineas = [l.strip() for l in f.readlines() if l.strip()]

# Parseo con formato estricto: datetime,status,mensaje
datos = []
for linea in lineas:
    try:
        timestamp = linea[:23]
        status = linea[26:31].strip()
        mensaje = linea[34:]
        datos.append((timestamp, status, mensaje))
    except Exception:
        continue

df_log = pd.DataFrame(datos, columns=["datetime", "status", "mensaje"])
df_log["datetime"] = pd.to_datetime(df_log["datetime"], errors="coerce")

# Mostrar ultimas lineas del log
print(f"Total de eventos en el log: {len(df_log)}")
display(df_log.tail(20))


Total de eventos en el log: 102


C:\Users\leant\AppData\Local\Temp\ipykernel_23764\1822835657.py:23: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df_log["datetime"] = pd.to_datetime(df_log["datetime"], errors="coerce")


,datetime,status,mensaje
82,2025-05-27 22:44:12.438,INFO,K - KKRS - 1030 filas procesadas.
83,2025-05-27 22:44:12.480,INFO,K - LPLA - 3651 filas procesadas.
84,2025-05-27 22:44:12.499,INFO,K - MCHPP - 45 filas procesadas.
85,2025-05-27 22:44:12.539,INFO,K - MORN - 5000 filas procesadas.
86,2025-05-27 22:44:12.572,INFO,K - MRNA - 1625 filas procesadas.
87,2025-05-27 22:44:12.600,INFO,K - OWL - 1118 filas procesadas.
88,2025-05-27 22:44:12.702,INFO,K - PCTY - 2815 filas procesadas.
89,2025-05-27 22:44:12.734,INFO,K - PEN - 2436 filas procesadas.
90,2025-05-27 22:44:12.773,INFO,K - PRMB - 5000 filas procesadas.
91,2025-05-27 22:44:12.813,INFO,K - PSTG - 2424 filas procesadas.
